In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import layers,Sequential

In [2]:
# 加载 ImageNet 预训练网络模型，并去掉最后一层
resnet = keras.applications.ResNet50(weights='imagenet',include_top=False)
resnet.summary()

94765736/94765736 [==============================] - 18s 0us/step

In [3]:
# 测试网络的输出
x = tf.random.normal([4,224,224,3])
out = resnet(x) # 获得子网络的输出
out.shape

TensorShape([4, 7, 7, 2048])

In [8]:
# 新建池化层
global_average_layer = layers.GlobalAveragePooling2D()
# 利用上一层的输出作为本层的输入，测试其输出
x = tf.random.normal([4,7,7,2048])
# 池化层降维，形状由[4,7,7,2048]变为[4,1,1,2048],删减维度后变为[4,2048]
out = global_average_layer(x)
out

<tf.Tensor: shape=(4, 2048), dtype=float32, numpy=
array([[-0.1217315 , -0.14568911, -0.218992  , ...,  0.10991631,
        -0.1024233 ,  0.00646547],
       [ 0.07370228,  0.19413666,  0.12961763, ...,  0.09410212,
        -0.18004282,  0.14002618],
       [ 0.02595836,  0.21693872,  0.20450312, ..., -0.2677715 ,
         0.21730831,  0.2397018 ],
       [ 0.04102283, -0.05869727, -0.1541958 , ..., -0.06757891,
         0.06789999,  0.10424548]], dtype=float32)>

In [9]:
# 新建全连接层
fc = layers.Dense(100)
# 利用上一层的输出[4,2048]作为本层的输入，测试其输出
x = tf.random.normal([4,2048])
out = fc(x) # 输出层的输出为样本属于 100 类别的概率分布
out

<tf.Tensor: shape=(4, 100), dtype=float32, numpy=
array([[-3.21041393e+00,  8.96453381e-01,  1.25465488e+00,
        -2.91373038e+00, -2.79335380e-01, -2.83102036e+00,
        -2.06872010e+00,  2.93093681e-01, -9.39393878e-01,
        -1.76762843e+00,  6.86492383e-01, -7.90473938e-01,
        -1.04956603e+00, -2.48624220e-01, -1.35144866e+00,
        -1.69907761e+00, -4.51949000e-01,  6.46555424e-03,
        -1.31806898e+00,  3.02144432e+00,  5.53680062e-02,
         6.18847489e-01, -1.51831329e-01, -1.14384711e-01,
         1.45098734e+00, -2.60081202e-01,  1.95647943e+00,
        -1.61847782e+00,  5.42414129e-01, -1.64850855e+00,
        -4.30866003e-01, -7.85426021e-01, -1.50864410e+00,
         1.77743316e+00,  1.50618911e+00, -1.18202806e-01,
        -1.63008547e+00,  1.77520841e-01,  1.38562727e+00,
        -1.11203671e+00,  1.72504044e+00,  7.49603331e-01,
        -2.60918140e-02, -7.52411544e-01,  1.09744930e+00,
         1.11395121e-02, -7.47133553e-01,  9.86412525e-01,
      

In [10]:
# 重新包裹成我们的网络模型
mynet = Sequential([resnet, global_average_layer, fc])
mynet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense (Dense)               (None, 100)               204900    
                                                                 
Total params: 23,792,612
Trainable params: 23,739,492
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
#通过设置 resnet.trainable = False 可以选择冻结 ResNet 部分的网络参数，只训练新建的
#网络层，从而快速、高效完成网络模型的训练。